# Método

O Random Forest (RF) é um algoritmo que combina várias "árvores de decisão" para chegar a resultados mais precisos e confiáveis. É como se várias pessoas votassem em uma decisão, e a maioria ganhasse. Aqui está o passo a passo para usá-lo:

**Prepare os dados:**

Junte os dados que você vai usar e limpe-os: resolva valores faltantes ou inconsistências.

Separe as colunas: quais são os fatores que influenciam (features) e qual é o objetivo (target).

Divida os dados em treino e teste, por exemplo, 80% para treino e 20% para teste.

**Configure os parâmetros:**

Decida quantas árvores o modelo terá (por exemplo, 100 árvores).

Escolha a profundidade máxima das árvores, para que elas não fiquem muito complexas.

Determine o número mínimo de exemplos necessários em cada folha da árvore.

**Treine o modelo:**

O algoritmo cria várias árvores, cada uma com um pedaço aleatório dos dados.

Cada árvore é treinada para fazer previsões e "vota" no resultado final.

**Teste e avalie:**

Use os dados de teste para verificar se o modelo funciona bem.

Meça a performance com métricas como precisão (accuracy), recall, ou erro médio (para problemas de regressão).

**Ajuste se necessário:**

Experimente mudar os parâmetros para tentar melhorar os resultados (isso pode ser feito com algo como Grid Search ou Random Search).

**Coloque em prática:**

Depois de ajustado, use o modelo para prever novos dados.

# Como entendo o RF?

O Random Forest (RF) é como perguntar para várias pessoas e seguir a opinião da maioria. Em vez de usar só uma decisão, eu uso várias “árvores de decisão” para chegar a um resultado mais confiável.

Como eu faço isso:

 Preparando os dados
Primeiro, junto todas as informações que preciso e arrumo os erros. Depois, separo o que influencia o resultado (features) e o que quero prever (target). Para garantir que o modelo aprenda bem, divido os dados: uma parte para treinar (80%) e outra para testar (20%).

**Configurando o modelo**

Escolho:

Quantas árvores usar (100 costuma ser um bom número).
O tamanho máximo das árvores, para não ficarem complicadas demais.
Quantos exemplos mínimos cada folha da árvore precisa ter.
Treinando o modelo aqui, o algoritmo cria várias árvores, cada uma aprendendo com partes diferentes dos dados. No final, elas “votam” e escolhem a melhor resposta.

Testando o modelo uso os dados de teste para ver se ele acerta. Para medir o desempenho, olho números como precisão (accuracy), recall ou erro médio (se for um problema de regressão).

Ajustando e melhorando se os resultados não forem bons, eu mudo algumas configurações para melhorar. Posso usar ferramentas como Grid Search ou Random Search para me ajudar.

Usando na prática quando tudo estiver certo, posso usar o modelo para fazer previsões sobre novos dados.

# Diferença entre Bagging e Random Forest

Bagging: Técnica geral que cria modelos independentes com amostras aleatórias dos dados. Funciona com qualquer tipo de modelo e combina resultados por média ou votação.

Random Forest: Uma aplicação específica do Bagging, usando exclusivamente árvores de decisão. Introduz aleatoriedade na escolha de features, tornando o modelo mais robusto e eficiente contra overfitting.

In [2]:
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Função personalizada para Random Forest
class RandomForest:
    def __init__(self, n_trees=10, max_features=None):
        self.n_trees = n_trees
        self.max_features = max_features
        self.trees = []

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        for _ in range(self.n_trees):
            X_sample, y_sample = self.bootstrap_sample(X, y)

            if self.max_features is None:
                self.max_features = int(np.sqrt(X.shape[1]))
            feature_indices = np.random.choice(X.shape[1], self.max_features, replace=False)

            tree = DecisionTreeClassifier()
            tree.fit(X_sample[:, feature_indices], y_sample)
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        tree_predictions = []
        for tree, feature_indices in self.trees:
            tree_predictions.append(tree.predict(X[:, feature_indices]))

        tree_predictions = np.array(tree_predictions).T
        majority_votes = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_predictions]
        return np.array(majority_votes)

# Carregar o dataset
data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Medir a acurácia para diferentes números de árvores
accuracies = []
n_trees_range = range(1, 50, 5)

for n_trees in n_trees_range:
    model = RandomForest(n_trees=n_trees, max_features=2)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracies.append(accuracy_score(y_test, predictions))

# Criar o gráfico interativo com Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(n_trees_range),
    y=accuracies,
    mode='lines+markers',
    name='Acurácia'
))

fig.update_layout(
    title='Performance do Random Forest com Diferentes Números de Árvores',
    xaxis=dict(title='Número de Árvores'),
    yaxis=dict(title='Acurácia'),
    template='plotly_dark'
)

fig.show()
